In [45]:
import pandas as pd

data = pd.read_csv("/home/ictc/Desktop/078msdsa016/covid_sentiment/CovidNepaliTweet_Research/sentiment_youtube_comment/collected_final_data.csv")

In [46]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

In [47]:
data.head()

,Unnamed: 0,text,label
0,1,1000 अंकले घटेको नेप्से 200 अंकले बढ्नु ठूलो क...,1
1,4,राम्रो भयो️️,1
2,6,जानकारीको लागि धन्यवाद रामहरी ब्रदर,1
3,18,"भारत-मधेस र नेपाल-चीन सम्बन्ध विग्रन्छ, मधेसी ...",0
4,25,"लेखनाथ न्यौपानेको खुलासा,महाधिबेशनमा एमसीसीको ...",0


In [48]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
#     import pdb;pdb.set_trace()
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [49]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Shushant/nepaliBERT')

def get_bert_embedding_sentence(sentence):

    query_embedding = model.encode(sentence)

    return query_embedding


No sentence-transformers model found with name /home/ictc/.cache/torch/sentence_transformers/Shushant_nepaliBERT. Creating a new one with MEAN pooling.
Some weights of BertModel were not initialized from the model checkpoint at /home/ictc/.cache/torch/sentence_transformers/Shushant_nepaliBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
get_bert_embedding_sentence("बजार ले जसरी ट्रेन्ड चेन्ज गर्यो यो हेर्दा")

array([-3.78136724e-01,  1.46421313e+00, -2.50940114e-01,  1.21089391e-01,
       -2.10015461e-01,  5.79064310e-01,  5.73764555e-02,  5.48238456e-01,
        8.93054679e-02,  1.26704082e-01,  5.64851046e-01,  5.09898424e-01,
       -1.54464722e-01,  1.09883189e+00, -2.29819983e-01,  9.90709305e-01,
       -1.57544613e-02,  2.34265506e-01,  2.91988552e-01,  6.28394127e-01,
       -1.01123124e-01, -2.04285771e-01,  7.81684101e-01, -3.33273619e-01,
       -6.46297157e-01, -5.77421129e-01,  4.73568857e-01,  2.96949670e-02,
       -1.38475299e-01,  2.41683632e-01,  2.15921775e-01,  9.17362198e-02,
        8.51969421e-02, -7.08448350e-01, -1.40382946e-01,  6.65720701e-02,
       -1.43700838e-02,  4.41497773e-01,  6.38163090e-02,  2.82624453e-01,
        3.26309383e-01,  8.53633881e-03,  4.68904078e-01, -3.83573711e-01,
        8.98801908e-03,  2.06401169e-01, -7.47826099e-01,  3.78553629e-01,
       -5.54051936e-01,  4.71117571e-02, -2.78353512e-01, -1.14877737e+00,
       -2.37238854e-01, -

In [52]:
data['text'].isnull().values.any()

False

In [53]:
data['word_embeddings'] = data['text'].apply(get_bert_embedding_sentence)

In [54]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [55]:
X,y = data['word_embeddings'], data['label']

In [56]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [57]:
svc = SVC(probability=True)

In [58]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [59]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [60]:
print(confusion_matrix(test_y, svc_pred))

[[426  89]
 [ 79 618]]


In [61]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.84      0.83      0.84       515
           1       0.87      0.89      0.88       697

    accuracy                           0.86      1212
   macro avg       0.86      0.86      0.86      1212
weighted avg       0.86      0.86      0.86      1212



In [62]:
accuracy_score(test_y, svc_pred)

0.8613861386138614

In [63]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [64]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, random_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.7367986798679867
[[229 286]
 [ 33 664]]
              precision    recall  f1-score   support

           0       0.87      0.44      0.59       515
           1       0.70      0.95      0.81       697

    accuracy                           0.74      1212
   macro avg       0.79      0.70      0.70      1212
weighted avg       0.77      0.74      0.71      1212



In [65]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(train_X.tolist(), train_y)



In [66]:
gbc_pred = gb_clf.predict(test_X.tolist())
print(accuracy_score(test_y, gbc_pred))

print(confusion_matrix(test_y, gbc_pred))

print(classification_report(test_y, gbc_pred))

0.83003300330033
[[407 108]
 [ 98 599]]
              precision    recall  f1-score   support

           0       0.81      0.79      0.80       515
           1       0.85      0.86      0.85       697

    accuracy                           0.83      1212
   macro avg       0.83      0.82      0.83      1212
weighted avg       0.83      0.83      0.83      1212



# Naive Bayes

In [67]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [68]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.7986798679867987
[[441  74]
 [170 527]]
              precision    recall  f1-score   support

           0       0.72      0.86      0.78       515
           1       0.88      0.76      0.81       697

    accuracy                           0.80      1212
   macro avg       0.80      0.81      0.80      1212
weighted avg       0.81      0.80      0.80      1212



# Ada Boost

In [69]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [70]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.8341584158415841
[[405 110]
 [ 91 606]]
              precision    recall  f1-score   support

           0       0.82      0.79      0.80       515
           1       0.85      0.87      0.86       697

    accuracy                           0.83      1212
   macro avg       0.83      0.83      0.83      1212
weighted avg       0.83      0.83      0.83      1212



# Decision Tree

In [71]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [72]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.7285478547854786
[[318 197]
 [132 565]]
              precision    recall  f1-score   support

           0       0.71      0.62      0.66       515
           1       0.74      0.81      0.77       697

    accuracy                           0.73      1212
   macro avg       0.72      0.71      0.72      1212
weighted avg       0.73      0.73      0.73      1212



# Extra Trees Classifier

In [73]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [74]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.863036303630363
[[412 103]
 [ 63 634]]
              precision    recall  f1-score   support

           0       0.87      0.80      0.83       515
           1       0.86      0.91      0.88       697

    accuracy                           0.86      1212
   macro avg       0.86      0.85      0.86      1212
weighted avg       0.86      0.86      0.86      1212



# Logistic Regression

In [75]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/ictc/anaconda3/envs/shushanta_nepali_gpt/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

In [76]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.8424092409240924
[[418  97]
 [ 94 603]]
              precision    recall  f1-score   support

           0       0.82      0.81      0.81       515
           1       0.86      0.87      0.86       697

    accuracy                           0.84      1212
   macro avg       0.84      0.84      0.84      1212
weighted avg       0.84      0.84      0.84      1212



# KNN

In [77]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [78]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.8110561056105611
[[393 122]
 [107 590]]
              precision    recall  f1-score   support

           0       0.79      0.76      0.77       515
           1       0.83      0.85      0.84       697

    accuracy                           0.81      1212
   macro avg       0.81      0.80      0.81      1212
weighted avg       0.81      0.81      0.81      1212



# MLP

In [79]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [80]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.8803630363036303
[[433  82]
 [ 63 634]]
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       515
           1       0.89      0.91      0.90       697

    accuracy                           0.88      1212
   macro avg       0.88      0.88      0.88      1212
weighted avg       0.88      0.88      0.88      1212

